## Resource-Constrained Project Scheduling Problem with Alternative Subgraphs

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Alternative Process Plan using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API. CP formulation of the problem is inspired by [Jakub Charvat bachelor thesis](https://dspace.cvut.cz/handle/10467/123532).

### Problem Definition

The Resource-Constrained Project Scheduling Problem with Alternative Subgraphs (RCPSP-AS) models a project as a directed acyclic graph $G=(N,A)$, where $N=\{1, \dots, n\}$ is the set of non-preemptive, topologically ordered activities $i, j$, and $A \subset N \times N$ defines the end-start precedence constraints $(i,j)$. Each activity $i$ has a predetermined duration $d_i$ and demands $r_{i,v}$ from a set of renewable resources $R$, where each resource $v \in R$ has a limited capacity $a_v$. The problem's core feature is a set of alternative subgraphs $L$, where each $l \in L$ (or $l' \in L$) is defined by a principal activity $p_l$ and a terminal activity $t_l$. Each subgraph $l$ contains multiple alternative branches $K_{p_l}$, and a branch $k \in K_{p_l}$ (or $k' \in K_{p_l}$) is a subset of activities $N_{b_k} \subset N^a$ starting from a unique branching activity $b_k$, such that $b_k \in N_{b_k} \wedge p_l \notin N_{b_k} \wedge t_l \notin N_{b_k}$ holds. This structure partitions $N$ into alternative activities $N^a = \bigcup N_{b_k}$ and fixed activities $N^f = N \setminus N^a$. Extensions include nested subgraphs and linked branches, the latter defined by a parameter $\kappa_{i,j}=1$ if an activity $i \in N_{b_k}$ is linked to $j \in N_{b_{k'}}$ (where $k \neq k'$).

### CPLEX Formulation

$$
\begin{aligned}
\min \quad
& \mathrm{endOf}(\mathrm{x}_{n-1})
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \mathrm{presenceOf}(\mathrm{x}_{0}) = 1,
\qquad &\qquad & \text{(2)} \\[1mm]
& \mathrm{ifThen}\!\left(\mathrm{presenceOf}(\mathrm{x}_i) \wedge \mathrm{presenceOf}(\mathrm{x}_j),\; \mathrm{endOf}(\mathrm{x}_i) \le \mathrm{startOf}(\mathrm{x}_j)\right),
\qquad & \forall (i,j)\in A
\quad & \text{(3)} \\[1mm]
& \sum_{k \in K_{p_l}} \mathrm{presenceOf}(\mathrm{x}_{k}) = \mathrm{presenceOf}(\mathrm{x}_{p_l}),
\qquad & \forall l \in L
\quad & \text{(4)} \\[1mm]
& \mathrm{presenceOf}(\mathrm{x}_i) = \left( \sum_{b_k \in M(B_i^{id})} \mathrm{presenceOf}(\mathrm{x}_{b_k}) > 0 \right),
\qquad & \forall i \in N \setminus \{0\}
\quad & \text{(5)} \\[1mm]
& \sum_{i \in N} \mathrm{pulse}(\mathrm{x}_i, r_{i,v}) \le a_v,
\qquad & \forall v \in R
\quad & \text{(6)} \\[1mm]
& \text{interval } \mathrm{x}_i\ \text{optional, size}=d_i,
\qquad & \forall i \in N
\quad & \text{(7)}
\end{aligned}
$$

**Objective:**

* **(1)** Minimize the makespan $C_{\max}$, defined as the end time of the sink activity $n-1$.

**Modeling Constraints:**

* **(2)** Ensures the source activity (activity 0) is always selected.
* **(3)** Enforces all precedence relations $(i,j) \in A$. The condition ($\mathrm{endOf}(\mathrm{x}_i) \le \mathrm{startOf}(\mathrm{x}_j)$) is enforced conditionally, only if *both* activities $i$ and $j$ are present.
* **(4)** Enforces subgraph branch selection. For each alternative subgraph $l$, the sum of the presence of its branching activities ($k \in K_{p_l}$) must equal the presence of its principal activity $p_l$.
* **(5)** Unified Activity Selection: Enforces that any activity $i \in N \setminus \{0\}$ is present if and only if at least one of the branches it belongs to is selected. This single rule covers both alternative activities (where a choice is made) and fixed activities (where presence is guaranteed by the dummy branch tied to the source $x_0$).
* **(6)** Enforces renewable resource limits. The `pulse` function models the resource demand $r_{i,v}$ during the execution of a present interval. The sum of all pulses on a resource $v$ must never exceed its capacity $a_v$.

**Variables:**

* **(7)** $\mathrm{x}_i$: An optional interval variable for each activity $i \in N$ with a fixed duration $d_i$.

#### Symbols and Notation

| Symbol / Function | Meaning | `docplex.cp` Reference |
| :--- | :--- | :--- |
| $N$ | Set of activities, indexed $0..n-1$ | |
| $A$ | Set of precedence constraints $(i,j)$ | |
| $L$ | Set of alternative subgraphs | |
| $R$ | Set of renewable resources | |
| $N^a, N^f$ | Sets of alternative and fixed activities | |
| $d_i$ | Predetermined duration of activity $i$ | |
| $r_{i,v}$ | Demand of activity $i$ for resource $v$ | |
| $a_v$ | Capacity of renewable resource $v$ | |
| $p_l$ | Principal activity of subgraph $l$ | |
| $K_{p_l}$ | Set of successors (branching activities) of $p_l$ | |
| $B_i^{id}$ | Set of branch IDs activity $i$ belongs to | |
| $M$ | Map from branch ID $B_i^{id}$ to branching activity | |
| $\mathrm{x}_i$ | Optional interval variable for activity $i$ | [docplex.cp.modeler.interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.interval_var) |
| $\mathrm{presenceOf}(\mathrm{x}_i)$ | $1$ if interval $\text{x}_{i}$ is present, else $0$ | [docplex.cp.modeler.presence_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.presence_of) |
| $\mathrm{endOf}(\dots)$ | End time of an interval | [docplex.cp.modeler.end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\mathrm{startOf}(\dots)$ | Start time of an interval | [docplex.cp.modeler.start_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.start_of) |
| $\mathrm{ifThen}(\dots)$ | Logical implication constraint | [docplex.cp.modeler.if_then](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.if_then) |
| $\mathrm{pulse}(\dots)$ | Time-varying resource usage | [docplex.cp.modeler.pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |

### CPLEX Implementation

#### Imports

In [1]:
from docplex.cp.model import CpoModel

# Using ascp package from Charvat's bachelor thesis
from ascp.load_instance import load_instance
from ascp.instance import AslibInstance, Subgraph

#### Reading the data file

In [2]:
instance = load_instance('../../data/rcpspas/robot-example/robot-example_a.rcp')

# create a dictionary to map activity IDs (0-based) to their objects
act_dict = {act.id: act for act in instance.activities}

# build branching activity map links branch_id to branching_activity_id
M = {
    branch_id: b_k_act_id
    for sub in instance.subgraphs if sub.principal_activity in act_dict
    for b_k_act_id in act_dict[sub.principal_activity].successors if b_k_act_id in act_dict
    for branch_id in act_dict[b_k_act_id].branches.intersection(sub.branches) if branch_id != 0
}
M[0] = 0

#### Create model and variables

In [3]:
# Create a CP Optimizer model
mdl = CpoModel(name=instance.name)

# (4) create optional interval variable for each activity i with a fixed duration d_i
x = {i: mdl.interval_var(
    name=f"T_{i}", optional=True, size=act.duration) for i, act in act_dict.items()}

#### Add constraints and define objective

In [4]:
# (1) Minimize the makespan (end time of the single sink activity)
mdl.add(mdl.minimize(mdl.end_of(x[len(instance.activities) - 1])))

# (2) Source activity is present
mdl.add(mdl.presence_of(x[0]) == 1)

# (3) Enforces all precedence relations
mdl.add(mdl.if_then(mdl.presence_of(x[act.id]) & mdl.presence_of(x[j]),
                    mdl.end_of(x[act.id]) <= mdl.start_of(x[j]))
        for act in instance.activities for j in act.successors if j in x)

# (4) Enforces subgraph branch selection
mdl.add(mdl.sum(mdl.presence_of(x[s])
                for s in act_dict[sub.principal_activity].successors if s in x) ==
        mdl.presence_of(x[sub.principal_activity])
        for sub in instance.subgraphs if sub.principal_activity in act_dict)

# (5) Enforces activity selection
mdl.add(mdl.presence_of(x[i]) == (mdl.sum(mdl.presence_of(x[M[b_id]])
                                          for b_id in act.branches if b_id in M) > 0)
        for i, act in act_dict.items() if i != 0)

# (6) Enforces renewable resource limits
mdl.add(mdl.sum(mdl.pulse(x[act.id], act.requirements[v])
                for act in instance.activities if act.requirements[v] > 0) <= capacity
        for v, capacity in enumerate(instance.resources) if capacity > 0)

#### Solve the model

In [5]:
print('Solving model...')
res = mdl.solve(FailLimit=100000,TimeLimit=10)
print('Solution: ')
res.print_solution()

Solving model...
 ! --------------------------------------------------- CP Optimizer 22.1.2.0 --
 ! Minimization problem - 12 variables, 31 constraints
 ! Presolve      : 6 extractables eliminated
 ! FailLimit            = 100000
 ! TimeLimit            = 10
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 43.7 (before), 43.7 (after)
 !  . Memory usage      : 549.0 kB (before), 549.0 kB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         12                 -
 + New bound is 11
 ! Using iterative diving.


 *           113       20  0.09s        1      (gap is 90.27%)
 *            21      110  0.09s        1      (gap is 47.62%)
              21      136          1    1   F         !presenceOf(T_5)
 + New bound is 21 (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 2 solutions found.
 ! Best objective         : 21 (optimal - effective tol. is 0)
 ! Best bound             : 21
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 42313
 ! Number of fails        : 15517
 ! Total memory usage     : 5.3 MB (5.3 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.09s (0.09s engine + 0.00s extraction)
 ! Search speed (br. / s) : 470144.4
 ! ----------------------------------------------------------------------------
Solution: 
-------------------------------------------------------------------------------
Model constraints: 31, variables: integer: 0, interval: 1

### Resource unit assignment

### Visualisation

## Aditional Resources

- [ASLIB dataset](https://www.projectmanagement.ugent.be/research/data)
- [Jakub Charvat bachelor thesis](https://dspace.cvut.cz/handle/10467/123532)